In [ ]:
## Полезная штука, если данных много и в аутпуте всё не помещается.
## В текущей директории создаётся тхт файл с полным содержимым.
#
#
#series_for_output = pd.Series(booking_df['negative_review'].value_counts().reset_index()['negative_review'].sort_values().str.strip().str.lower().unique())
#
## Сюда нужно поместить то, что хотим полностью увидеть.
#output_data = series_for_output[series_for_output.apply(lambda x: len(x)) < 14]

## Если аутпут - сериес:
#with open('output_neg_rev.txt', 'w') as f:
#    output_data.to_csv('output_neg_rev.txt', sep='\t', index=False)

## Если аутпут - списко:
#with open('output.txt', 'w') as f:
#    for item in output_data:
#        f.write("%s\n" % item)

<h2 id='contents'>Оглавляшка:</h2>

1. [Функция-аналог .info() - my_info()](#myinfo)
2. Графические функции:
    * [Функция рисования 3-х графиков: гистограммы, коробки и Q-Q в одном флаконе.](#histboxqq)(для массива чисел)
    * [Функция рисования графика корреляции.](#mycorr)
    * [Функция декорирования оси Х столбчатой диаграммы.](#decoratexlabels)(крутая штука)
3. [Функция, создающая словарь для .fillna() заполнения NaN.](#fillnadict)
4. [Функция отлова выбросов по методу Тьюки.](#outlierstukie)
5. [Функция проверки на нормальность Шапиро-Уилка.](#shapirowilkfunc)
6. [Функция проверки равенства дисперсий от Левена.](#levenefunc)
7. Функция для работы с ["строками", "внутри", "списка тэгов", "1 Adult", "2 Adults"]
    * [ОСНОВНАЯ](#maintagsworkfunc)
    * [Второстепенная](#retagsworkfunc)
8. Функция для работы с текстом. Когда столбец состоит только из элементов str
    * [Основная функция с алгоритмом Вейдера](#vadersentanalmain)
    * [Второстепенная функция с алгоритмом Вейдера](#vadersentanalsec)
    * [Функция с алгоритмом Роберты на основе отзывов в твиттере](#robertapolarity)
9. [Функция для тренировки нескольких моделей, используя несколько алгоритмов ML за 1 заход](#multiplemodelstraining)
10. [Функция для генерации комбинаций элементов входящего списка](#combinationsgenerator) (классный комбинатор элементов списка)
11. [Функция для тестирования силы столбца, относительно целевого. f_classif + chi2](#fclassifchitester)
12. [Группировщик с применением ко всему датафрейму](#specialgrouper)
13. Тестовые функции:
    * [Функция для удаления сильно скоррелированных столбцов. (тестовая)](#highcorreraser)
    * [Функция для автоматического поиска сильных столбцов относительно целевого столбца (тестовая)](#strongcolssearcher)
    * ТЕСТОВАЯ [Функция для определения положительных и отрицательных слов в предложении](#goodbadwordssearcher) (супер сырая)

Пример работы tqdm

In [2]:
from tqdm.notebook import tqdm
import time

list_for_tqdm = [i for i in range(100)]

total_progress = 100
with tqdm(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s]') as pbar:
    cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100.
##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
# сюда нужно вставить свой цикл.
    for i, word in enumerate(list_for_tqdm):
        time.sleep(0.1)
        
##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############        
        # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
        remaining_iterations = len(list_for_tqdm) - i # вот тут пригодился индекс i
        remaining_progress = total_progress - cumulative_progress
        step_size = remaining_progress / remaining_iterations
        
        # Обновляем общий прогресс.
        cumulative_progress += step_size
        pbar.update(step_size)

Cell progress: : 0%|          | 0/100 [Elapsed: 00:00 m/s]

<h2 id='myinfo'>Функция печатающая аналог <span style='color:orange'>.info()</span> в расширенном варианте.</h2>

In [ ]:
import pandas as pd

# My info.
def my_info(df, catmaxnum=None):
    """Info function to show basic df indicators

    Args:
        df (pd.Dataframe): any df.
        catmaxnum (int): the minimum threshold(rus: порог) after which a column will be considered a category. Defaults to None.

    Returns:
        _type_: pd.Dataframe
    """
    df_copy = df.copy() # it needs here.
    
    # Loop and condition for moment, when there are unhashable types in our column. (list, dict, set)
    for i in df_copy.columns:
        if any(isinstance(cell, (list, set, dict)) for cell in df_copy[i]):
            df_copy[i] = df_copy[i].astype(str)
                
    func_df = pd.DataFrame({
        
        'column': df_copy.columns,
        'num of unique vals': df_copy.nunique(),
        'type': [str(df_copy.dtypes[i]) for i in df_copy.columns],
        'mode': [df_copy[i].mode()[0] for i in df_copy.columns],
        'number of entries': len(df_copy),
        'NaN vals': df_copy.isna().sum(),
        'number of dublics': df_copy.duplicated().sum(),
        'describe': [{'min': df_copy[i].min(), 'max': df_copy[i].max(), 'mean': round(df_copy[i].mean()), 'std': round(df_copy[i].std())} if np.issubdtype(df_copy[i].dtype, np.number) else 'type obj' for i in df_copy.columns]
        
    }).sort_values(by='num of unique vals').reset_index(drop=True)
    
    if catmaxnum:
        func_df.insert(3, 'classification', ['category' if i < catmaxnum else 'numeric' for i in func_df['num of unique vals']])
        
    return func_df

[to contents](#contents)

<h2 id='outlierstukie'>Функция отлова выбросов по методу Тьюки</h2>

In [ ]:
import pandas as pd
import numpy as np

# Tukie outliers finder function.
def outliers_iqr_mode_log(data, feature=None, log_scale=False, left=1.5, right=1.5):
    
    func_data = data.copy()
    
    if isinstance(func_data, pd.DataFrame):    
        if log_scale: # if we want to logarithm a feature.
            if any(func_data[feature] == 0):
                x = np.log(func_data[feature] + 1)
            else:
                x = np.log(func_data[feature])
        else:
            x = func_data[feature]
    else:
        x = func_data # if data == array or series
        
    quantile_25, quantile_75 = x.quantile(0.25), x.quantile(0.75)
    iqr = quantile_75 - quantile_25
    
    lower_bound = quantile_25 - iqr * left
    upper_bound = quantile_75 + iqr * right
    
    outliers = func_data[(x < lower_bound) | (x > upper_bound)]
    cleaned = func_data[(x >= lower_bound) & (x <= upper_bound)]
    
    return outliers, cleaned.reset_index(drop=True) # .copy() is needed here if we wanna use cleaned daataframe. Or system will swear.

[to contents](#contents)

<h3 id='histboxqq'>Функция рисования 3-х графиков: гистограмма, коробка и Q-Q.</h3>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm # рисовалка qq-plot.
import seaborn as sns

def hist_box_qq(arg, full_iqr=False):
    """Function of drawing histogram, box and cuckoo in one bottle.

    full_iqr... It's shorter.. I wrote a function for the lognormal distribution, i.e. there are no outliers on the left.
    But then I thought about it and decided to add it, because then I still log the feature.
    And in general, if the array is usually distributed.
    In a word: full_iqr is Boolean. By default, False. I.e., only the right borders are drawn.
    
    The input is a Series or array.

    Args:
        arg (Series/np.array): any numeric series or array.
        full_iqr (bool, optional): choose, u wanna show only right outliers borders or both. Defaults to False.

    Returns:
        plt
    """
    func_arg = arg.copy()
    # replace inf.
    func_arg = func_arg.replace([np.inf, -np.inf], np.nan)
    # If there is a nan, I drop it. Because it does not draw if there is a nan.
    func_arg = pd.Series(func_arg).dropna()
    
    fig, ax = plt.subplots(ncols=3, nrows=1, figsize=[18, 5])
    
    # The histogram. And the settings for the histogram lines. Legend = label in each line.
    # Red is the median, green is the tukey, and blue is 3 sigma.
    sns.histplot(func_arg, kde=True, bins=50, ax=ax[0])
    
    ax[0].get_lines()[0].set_color('black')
    ax[0].axvline(func_arg.median(), color='red', linestyle='--', linewidth='1.8', label='median')
    ax[0].axvline(func_arg.quantile(0.75) + ((func_arg.quantile(0.75) - func_arg.quantile(0.25)) * 1.5), color='g', ls='--', lw=2, label='1.5 IQR Tukie')
    ax[0].axvline(func_arg.mean() + 3 * func_arg.std(), color='b', ls='--', lw=2, label='3 IQR z-score')
    ax[0].legend()
    ax[0].set_xlabel(f'Feauture {func_arg.name}')
    
    # Box. And the settings for it.
    sns.boxplot(func_arg, ax=ax[1], orient='h', medianprops={'color': 'red', 'linestyle': '--'})

    ax[1].axvline(func_arg.quantile(0.75) + ((func_arg.quantile(0.75) - func_arg.quantile(0.25)) * 1.5), color='g', ls='--', lw=2, label='1.5 IQR Tukie')
    ax[1].axvline(func_arg.mean() + 3 * func_arg.std(), color='b', ls='--', lw=2, label='3 IQR z-score')
    ax[1].legend()    
    ax[1].set_xlabel(f'Feauture {func_arg.name}')
    
    if full_iqr: # Left emission catch lines are added.
        ax[0].axvline(func_arg.quantile(0.25) - ((func_arg.quantile(0.75) - func_arg.quantile(0.25)) * 1.5), color='g', ls='--', lw=2)
        ax[0].axvline(func_arg.mean() - 3 * func_arg.std(), color='b', ls='--', lw=2)
        ax[1].axvline(func_arg.quantile(0.25) - ((func_arg.quantile(0.75) - func_arg.quantile(0.25)) * 1.5), color='g', ls='--', lw=2)
        ax[1].axvline(func_arg.mean() - 3 * func_arg.std(), color='b', ls='--', lw=2)
    
    # This line builds a Q-Q graph.
    qq = sm.ProbPlot(func_arg, fit=True).qqplot(marker='*', markerfacecolor='b', markeredgecolor='b', alpha=0.3, ax=ax[2])
    # Line, also with the ability to influence color. fmt parameter.
    sm.qqline(qq.axes[2], line='45', fmt='r', linestyle='--')
    
    # The general title.
    plt.suptitle(f'Distribution of the feature: {func_arg.name}').set_fontsize(20)
    
    return plt

[to contents](#contents)

<h3 id='mycorr'>Функция рисования тепловой карты корреляции.</h3>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

def my_corr(df, method='spearman', corrneg=0, corrpos=0, figsize=(10, 5), xtickrot=90, show_high_corr_cols_names=False):
    """
    The function of drawing a correlation graph, but not an ordinary one, but one in which the lower part is the entire one,
    and the upper part contains only those correlation values that are greater and less than some number we have specified.

    Args:
        df (pd.Dataframe): any df.
        method (str): method u like. default spearman.
        corrneg (float): negative threshold for display. (for negative correlation value)
        corrpos (float): positive threshold. (for a positive value) both default to 0. At 0, the full standard correlation matrix will be displayed.
        figsize (tuple): u know what it is.)
        xtickrot(int): rotate x axis labels. default 90
        show_high_corr_cols_names(bool): if we wanna watch names of high corr cols. default False

    Returns:
        _type_: plt or df
    """
    corr_df = df.corr(numeric_only=True, method=method).copy()
    
    # We find indices where the correlation value is greater and less than a certain number.
    high_corr_pairs = ((corr_df >= corrpos) & (corr_df <= 1)) | (corr_df <= corrneg)
    
    # We receive only those pairs of features where the condition is met. The rest are NaN.
    high_corr_features = corr_df[high_corr_pairs]
    
    # if we wanna watch high corr pairs in the form of a df. Without picture. I decided 2 split it up.
    if show_high_corr_cols_names:
        # stack hight corr feature.
        shccn = high_corr_features.stack().to_frame().reset_index().rename(columns={0: 'corr'})
        # sort'em.
        shccn[['level_0', 'level_1']] = np.sort(shccn[['level_0', 'level_1']], axis=1)
        # keep only 1 pair of high corr cols names.
        shccn = shccn.groupby(['level_0', 'level_1']).first().reset_index()
        # removing the diagonal.
        shccn = shccn.query('level_0 != level_1')
        # return with resetting index.
        return shccn.sort_values(by='corr', ascending=False).reset_index(drop=True)
    
    # if we wanna see the pic.
    else:
        fig, ax = plt.subplots(figsize=figsize)
    
        plt.suptitle(f'Correlation of incoming dataframe features.')
        
        # 2 masks, one triu (tri upper), the second tril (tri lower).
        mask_for_logging_df = np.triu(np.ones_like(corr_df, dtype=bool))
        mask_for_logging_df_2 = np.tril(np.ones_like(high_corr_features, dtype=bool))
        
        # 2 graphics. I didn’t figure out how to make such a picture using 1 chart. That's why cbar=False helps.
        sns.heatmap(corr_df, annot=True, cmap='coolwarm', mask=mask_for_logging_df, vmax=1, vmin=-1, linewidths=0.1, fmt='.2f') # <--- lower part
        
        # if condition == 1 high_corr_features contains only nan, so we dont need upper part. 
        # or.. high_corr_features has no nan, i.e. corrpos and corrneg == 0, so we draw all corr matrix.
        if (len(high_corr_features.isna().sum().value_counts()) != 1) | ((corrpos == 0) & (corrneg == 0)):
            sns.heatmap(high_corr_features, annot=True, cmap='coolwarm', mask=mask_for_logging_df_2, cbar=False, vmax=1, vmin=-1, linewidths=0.1, fmt='.2f') # <--- upper part
        
        ax.set_xticklabels(ax.get_xticklabels(), rotation=xtickrot)
    
        plt.plot([0, len(df.columns)], [0, len(df.columns)], color='black', linewidth=2); # It's just a line. I don't know why. But i like it.
        
        return plt

[to contents](#contents)

<h3 id='fillnadict'>Функция, создающая словарь для заполнения метода .fillna</h3>

In [ ]:
def fillna_dict_for_custom_df(df, func):
    """Function for generating a dictionary for the fillna function argument.
    
    The function takes as input a dataframe with np.nan and the function that we want to aggregate.
    That is, we have a feature in which there is a nan. We want to fill the nan, for example, with the median.
    We run through the names of the columns that have nan and assign each nan a median value for the column.
    Why the condition? When we ask for mode, we get a Series object that needs to be accessed by index to get the value.
    And when we ask for the average or median, it immediately returns the value.

    Args:
        df (pd.Dataframe): any df
        func (str): any func in str format. 
        
        Ex: df.fillna(value=fillna_dict_for_custom_df(df, 'mean'))

    Returns:
        _type_: dict
    """
    return {i: df[i].agg(func)[0] if func == 'mode' else df[i].agg(func) for i in df.loc[:, df.isna().mean() > 0].columns}

[to contents](#contents)

<h3 id='decoratexlabels'>Функция декорирования оси Х столбчатой диаграммы.</h3>

In [ ]:
## ОРИГИНАЛ!
#from itertools import groupby # для рисования мультииндекс меток по оси х (и не только, но здесь - только).
#
## Чумовейшие три функции, которые рисуют мультииндекс разметку по оси Х!!!
## Работа происходит по третьей функции. 
## На вход подаётся ax и датафрейм вида: 
## 1. главный индекс; 
## 2. второстепенный индекс или группа индексов, по иерархии от старшего к младшему; 
## 3. последним идёт индекс, который является оттенком (hue).
## Вид группировки: .groupby(['1_st_order_idx', '2_nd_order_idx', ..., 'hue_idx'])['nums_column'].mean().unstack()
## P.S. для работы нужен groupby из itertools
#
#def add_line(ax, xpos, ypos):
#    line = plt.Line2D([xpos, xpos], [ypos + .1, ypos],
#                      transform=ax.transAxes, color='gray')
#    line.set_clip_on(False)
#    ax.add_line(line)
#
#def label_len(my_index,level):
#    labels = my_index.get_level_values(level)
#    return [(k, sum(np.fromiter((1 for i in g), dtype=int))) for k, g in groupby(labels)]
#
#def label_group_bar_table(ax, df):
#    ypos = -.1
#    scale = 1./df.index.size
#    for level in range(df.index.nlevels)[::-1]:
#        pos = 0
#        for label, rpos in label_len(df.index, level):
#            lxpos = (pos + .5 * rpos)*scale
#            ax.text(lxpos, ypos, label, ha='center', transform=ax.transAxes)
#            add_line(ax, pos*scale, ypos)
#            pos += rpos
#        add_line(ax, pos*scale ,ypos)
#        ypos -= .1
        
def label_group_bar_table(ax, df):
    """label_group_bar_table 

    Чумовейшие три функции, которые рисуют мультииндекс разметку по оси Х!!!
    На вход подаётся ax и датафрейм вида: 
     1. главный индекс; 
     2. второстепенный индекс или группа индексов, по иерархии от старшего к младшему; 
     3. последним идёт индекс, который является оттенком (hue).
     Вид группировки: .groupby(['1_st_order_idx', '2_nd_order_idx', ..., 'hue_idx'])['nums_column'].mean().unstack()
     P.S. для работы нужен groupby из itertools
     P.S.2 Сие ваяние не моё, потому хз, что там происходит и я не распишу каждую строчку, но результат меня устраивает.)

    Args:
        ax (_type_): ax with ur plot.
        df (_type_): ur df which u wanna plot.
    """
    from itertools import groupby # для рисования мультииндекс меток по оси х (и не только, но здесь - только).
    
    def add_line(ax, xpos, ypos):
        line = plt.Line2D([xpos, xpos], [ypos + .1, ypos],
                          transform=ax.transAxes, color='gray')
        line.set_clip_on(False)
        ax.add_line(line)
    
    def label_len(my_index,level):
        labels = my_index.get_level_values(level)
        return [(k, sum(np.fromiter((1 for i in g), dtype=int))) for k, g in groupby(labels)]
    
    
    ypos = -.1
    scale = 1./df.index.size
    for level in range(df.index.nlevels)[::-1]:
        pos = 0
        for label, rpos in label_len(df.index, level):
            lxpos = (pos + .5 * rpos)*scale
            ax.text(lxpos, ypos, label, ha='center', transform=ax.transAxes)
            add_line(ax, pos*scale, ypos)
            pos += rpos
        add_line(ax, pos*scale ,ypos)
        ypos -= .1

####################################################################################################################
# Пример использования:

fig, ax = plt.subplots(figsize=(30, 8))

plt.suptitle('Динамика соотношения среднего годового дохода в зависимости от формата занятости, года и опыта').set_fontsize(25)

# Подготовка датафрейма.
df_for_this_cell = df_for_this_cell.groupby([choose_job, 'remote_ratio', 'work_year', 'experience_level'])['salary_in_usd'].mean().unstack()

# Сама строка.
df_for_this_cell.plot(kind='bar', ax=ax, stacked=True, legend=True)

ax.set_xticklabels('') # обязательно удаление названий по оси х
ax.set_xlabel('')

# И оборачивание подготовленного, сгруппированного датафрейма в функцию
# На самом деле можно и без этого. Функция чисто декоративная.
label_group_bar_table(ax, df_for_this_cell)
####################################################################################################################

[to contents](#contents)

<h3 id='shapirowilkfunc'>Функция проверки на нормальность Шапиро-Уилка.</h3>

In [ ]:
import pandas as pd
import scipy.stats as stats

alpha = 0.5

# Функция проверки на нормальность Шапиро-Уилка.
def shapirowilk(df_or_arr, alpha=0.05):
    """Продвинутая функция проверки нормальности Шапиро-Уилка.
    
    Тут всё просто. Если датафрейм, то формируем через цикл списки названий нормальных и не нормальных столбцов.
    Иначе сразу печатаем реультат теста. Т.к. у нас Series или массив.
    Далее смотрим на длины списков и формируем список выводов функции. И выводим его.
    Почему продвинутая? Потому что я раньше считал количество норм и не норм столбцов через: dict(Counter([shapirowilk(df_for_this_cell[i]) for i in df_for_this_cell.columns]))
    а сама функция выглядела, как функция Левена ниже.))

    Args:
        df_or_arr (pd.DataFrame): Dataframe or Series or array
        alpha (float): alpha significance level

    Returns:
        df or str
    """
    func_df = pd.DataFrame()

    if isinstance(df_or_arr, pd.DataFrame):
        for i, j in enumerate(df_or_arr.columns):
            if stats.shapiro(df_or_arr[j])[1] <= alpha:
                func_df.loc[i, 'Distribution'] = df_or_arr[j].name
                func_df.loc[i, 'Status'] = 'не нормальное'
            else:
                func_df.loc[i, 'Distribution'] = df_or_arr[j].name
                func_df.loc[i, 'Status'] = 'нормальное'
    else:
        return 'Распределение не нормальное. При alpha = 0.05' if stats.shapiro(df_or_arr)[1] <= alpha else 'Распределение нормальное. При alpha = 0.05'
    
    return func_df

[to contents](#contents)

<h3 id='levenefunc'>Функция проверки равнества дисперсий от Левена.</h3>
(недоработанная)

In [ ]:
# Функция проверки на равенство дисперсий от Левена.
def levene(*args):
    return 'Дисперсии не равны. Нужно использовать непараметрический тест. При alpha = 0.05' if stats.levene(*args)[1] <= alpha else 'Дисперсии равны. При alpha = 0.05'

[to contents](#contents)

<h3 id='maintagsworkfunc'>Функция для работы с ["строками", "внутри", "списка тэгов", "1 Adult", "2 Adults"]</h3>
ОСНОВНАЯ.

In [ ]:
def transform_specific_string(tags_column, words_to_transform, list_of_exceptions=[], hard_split_list=[], replacement_dict=False):
    from tqdm.notebook import tqdm_notebook
    """
    Функция преобразования str элементов списка.
    Типа когда у нас есть список, в котором есть строки, которые нужно побить по какому-то слову или словосочетанию.
    И вернуть обратно такой же список, но с побитыми строками. 
    P.S. Почему не сделать изначально весь столбец .lower() и потом использовать функцию?
    Приятней же искать в выводе файла 'output_transformed.txt' следующие слова на обрезку, которые начинаются с заглавной буквы.
    Ну, лично мне. Можно изначально сделать всё lower и оно всё равно будет работать.

    Args:
        tags_column (series): столбец со списками, которые с элементами str.
        split_word (str): ключевое слово или словосочетание.
        list_of_exceptions (list): список исключений, которые мы не хотим видеть в итоговом списке.
        hard_split_list (list): список слов, по которым строка будет делиться, не склеивая остатки (with, and, etc).
        replacement_dict (dict): словарь замены одинакового, написанного по-разному.

    Returns:
        list: возвращаем побитый список.))
        
    Пример использования: 
    нужно вставить код ниже в ячейку после функции и поменять your_df[tags_col] на свой дф с столбцом тэгов и везде заменить your_df (выделить your_df и на клаве: ctrl+D или cmd+D если мак). 
    ----------------------------------------------------------------------------------------------------------
        words_to_transform = [# любые слова или словосочетания, по которым хотим побить каждую строку, порядок важен] # типа сначала сложносоставные слова "1 2 Adult", а потом "2 Adult". У себя я разбил на 2 тего: "1 2 Adult" - 1 адулт и 2 адулт. Чувак же указал.
        list_of_exceptions = [# слова исключения, которым в списках не место (with, and)] # Тут порядок не важен.
        hard_split_list = [# слова, по которым строка будет делиться, без склеивания остатков (with, and, etc)]
        replacement_dict = {# словарь, если хотим заменить какие-то слова или словосочетания} # И тут тоже.
        
        # Помещаем преобразование в отдельный столбец.
        your_df['transformed'] = transform_specific_string(your_df[tags_col], words_to_transform=words_to_transform, list_of_exceptions=list_of_exceptions, replacement_dict=replacement_dict)
        
        # vvvvvvvvvvvvvvvvvvv  Ниже работа с выводом  vvvvvvvvvvvvvvvvvvv
        # Если хотим посмотреть, что у нас получилось используем следующее:
        # Полезная штука, если данных много и в аутпуте всё не помещается.
        # В текущей директории создаётся файл в котором находится список уникальных элементов столбца. Ну или не совсем уникальных, если не делали .lower() Нужно вставить после .explode()
        output_data = list(set(your_df['transformed'].explode()))
        
        with open('output_transformed.txt', 'w') as f:
            for item in output_data:
                f.write("%s\n" % item)
        
        # Можно проверить, как оно бьётся по конкретному слову, словосочетанию:
        print(your_df.loc[[' 2 adult '.lower() in str(i) for i in your_df['tags'].apply(lambda x: [i.lower() for i in x])]])
        
        # Можно вывести 2 столбца со списками и посмотреть, как идут дела.)
        your_df.loc[:, ['tags', 'transformed']]
        
        # Если есть ещё одна функция, которая делает то же самое, то можно проверить столбцы на не равенство после работы двух функций. 
        # Всё должно быть False. И если есть True, убираем вальюкаунтс и спереди добавляем df.loc[]. И смотрим, что где плохо бьётся.
        (df_main['transformed'].apply(lambda x: [i.lower() for i in x]) != df_main['transformed_2'].apply(lambda x: [i.lower() for i in x])).value_counts()
    ----------------------------------------------------------------------------------------------------------
    """
    # Внутренняя функция, которая лупит конкретную строку.
    def transform(row, split_word, list_of_exceptions, hard_split_list, replacement_dict):
        # Возвращаемый список.
        new_row = []
        # Цикл по элементам строки.
        for item in row:
            # Создаём временную переменную, которая является элементом списка + пробелы с двух сторон, потому что реплейс мы делаем по отдельно стоящему слову, т.е. слову, которое окружено с двух сторон пробелами.
            temp_item = ' ' + item.lower() + ' '
            # Условие на вхождение слова разделителя в элемент списка. Я сначала сделал через re.search, но re работает дольше, потому переписал через вот так.
            if split_word.lower() in item.lower():
                # Если это слова, по которым нужно именно разделить на отдельные части, то:
                if split_word.lower() in hard_split_list:
                    # Заменяем слово разделитель на |||||
                    temp_item = temp_item.replace(f' {split_word.lower()} ', '|||||')
                    # Если реплейс сработал то:
                    if '|||||' in temp_item:
                        # Делим по реплесу.
                        split_items = temp_item.split('|||||')
                        # Добавляем само слово разделитель и остатки после разделения - отдельно.
                        new_row.extend([split_word.strip(), *[i.strip() for i in split_items]])
                    else:
                        # Иначе записываем просто итем.
                        new_row.append(item.strip())
                # Если же это другие слова или словосочетания, то:
                else:
                    # Заменяем разделитель на |||||.
                    temp_item = temp_item.replace(f' {split_word.lower()} ', '|||||')
                    # Тут как раз проверка на "сработал реплейс или нет". 
                    # У меня был момент: входящая строка: 'Two Connecting Superior Double Rooms', слово-разделитель: 'Superior Double Room'. 
                    # Входит? Входит. Реплейсит? - нет. Ничего не возвращает и я добавляю просто слово разделитель, упуская саму строку. 
                    # Потому сначала вот это условие.
                    if '|||||' in temp_item:
                        # Если да, то бьём и
                        split_items = temp_item.split('|||||')
                        # добавляем само слово разделитель, и склеиваем остатки по пробелу.
                        new_row.extend([split_word.strip(), ' '.join([i.strip() for i in split_items])])
                    else:
                        # Иначе записываем просто итем.
                        new_row.append(item.strip())
            else:
                # Как и здесь.
                new_row.append(item.strip())
                
        # Чистим список.
        new_row = [item for item in new_row if item != '' and item not in list_of_exceptions]
        # Переименовываем элементы списка только тогда, когда это нужно и переименование зависит от слова-разделителя.
        if replacement_dict and split_word in replacement_dict:
            new_row = [replacement_dict[split_word] if split_word in item else item for item in new_row]

        return new_row
    
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############
    # Общий прогресс прогресс бара от 0 до 100% и Инициализация прогресс бара с настройками отображения.
    total_progress = 100
    with tqdm_notebook(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s, Remaining: {remaining} m/s]') as pbar:
        cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100.
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
        
        # Здесь сам цикл, в котором мы перебираем каждое слово и передаём его в каждую строку, которая является списком.))
        # И обрабатываем это слово с помощью функции transform для каждой строки.
        for i, word in enumerate(words_to_transform):
            tags_column = tags_column.apply(transform, split_word=word, list_of_exceptions=list_of_exceptions, hard_split_list=hard_split_list, replacement_dict=replacement_dict)
            
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############        
            # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
            remaining_iterations = len(words_to_transform) - i # вот тут пригодился индекс i
            remaining_progress = total_progress - cumulative_progress
            step_size = remaining_progress / remaining_iterations
            
            # Обновляем внутренний и общий прогресс.
            cumulative_progress += step_size
            pbar.update(step_size)
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############

    return tags_column

[to contents](#contents)

<h3 id='retagsworkfunc'>Функция для работы с ["строками", "внутри", "списка тэгов", "1 Adult", "2 Adults"]</h3>
Второстепенная с использованием модуля re.

In [ ]:
def transform_specific_string_2(tags_column, words_to_transform, list_of_exceptions=[], replacement_dict=False): # Рабочий вариант №2
    from tqdm.notebook import tqdm_notebook
    import re
    """
    Функция преобразования str элементов списка.
    Типа когда у нас есть список, в котором есть строки, которые нужно побить по какому-то слову или словосочетанию.
    И вернуть обратно такой же список, но с побитыми строками. 
    P.S. Почему не сделать изначально весь столбец .lower() и потом использовать функцию, но.. 
    Приятней же искать в выводе файла 'output_transformed.txt' следующие слова на обрезку, которые начинаются с заглавной буквы.
    Ну, лично мне. 

    Args:
        tags_column (series): столбец со списками, которые с элементами str.
        split_word (str): ключевое слово или словосочетание.
        list_of_exceptions (list): список исключений, которые мы не хотим добавлять при сплите: with, and.
        replacement_dict (dict): словарь замены одинакового, написанного по-разному.

    Returns:
        list: возвращаем побитый список.))
        
    Пример использования: 
    нужно вставить код ниже в ячейку после функции и поменять your_df[tags_col] на свой дф с столбцом тэгов и везде заменить your_df (выделить your_df и на клаве: ctrl+D или cmd+D если мак). 
    ----------------------------------------------------------------------------------------------------------
        words_to_transform = [# любые слова или словосочетания, по которым хотим побить каждую строку, порядок важен] # типа сначала сложносоставные слова "1 2 Adult", а потом "2 Adult". У себя я разбил на 2 тего: "1 2 Adult" - 1 адулт и 2 адулт. Чувак же указал.
        list_of_exceptions = [# слова исключения, которым в списках не место (with, and)] # Тут порядок не важен.
        replacement_dict = {# словарь, если хотим заменить какие-то слова или словосочетания} # И тут тоже.
        
        # Помещаем преобразование в отдельный столбец.
        your_df['transformed'] = transform_specific_string(your_df[tags_col], words_to_transform=words_to_transform, list_of_exceptions=list_of_exceptions, replacement_dict=replacement_dict)
        
        # vvvvvvvvvvvvvvvvvvv  Ниже работа с выводом  vvvvvvvvvvvvvvvvvvv
        # Если хотим посмотреть, что у нас получилось используем следующее:
        # Полезная штука, если данных много и в аутпуте всё не помещается.
        # В текущей директории создаётся файл в котором находится список уникальных элементов столбца. Ну или не совсем уникальных, если не делали .lower() Нужно вставить после .explode()
        output_data = list(set(your_df['transformed'].explode()))
        
        with open('output_transformed.txt', 'w') as f:
            for item in output_data:
                f.write("%s\n" % item)
        
        # Можно проверить, как оно бьётся по конкретному слову, словосочетанию:
        print(your_df.loc[[' 2 adult '.lower() in str(i) for i in your_df['tags'].apply(lambda x: [i.lower() for i in x])]])
        
        # Можно вывести 2 столбца со списками и посмотреть, как идут дела.)
        your_df.loc[:, ['tags', 'transformed']]
        
        # В эту версию я добавлю отладчик, т.к. основная версия другая, без модуля re.
        new_row = []
        item = 'Two Connecting Superior Double Rooms'
        split_word_list = ['Superior Double Room']
        
        for split_word in split_word_list:
            temp_item = ' ' + item.lower() + ' '
            
            if split_word.lower() in item.lower():
                if split_word.lower() == 'with' or split_word.lower() == 'and':
                    temp_item = temp_item.replace(f' {split_word.lower()} ', '|||||')
                    split_items = temp_item.split('|||||')
                    new_row.extend([split_word.strip(), *[i.strip() for i in split_items]])
                else:
                    
                    temp_item = temp_item.replace(f' {split_word.lower()} ', '|||||')
                    
                    if '|||||' in temp_item:
                        print(temp_item) # Принтами можно проверить каждый шаг.
                        split_items = temp_item.split('|||||')
                        new_row.extend([split_word.strip(), ' '.join([i.strip() for i in split_items if len(i) > 1])])
                    else:
                        new_row.append(item.strip())
            else:
                new_row.append('xyu')
        new_row
        
        # Проверяем на равенство после работы двух функций.
        (df_main['transformed'].apply(lambda x: [i.lower() for i in x]) != df_main['transformed_2'].apply(lambda x: [i.lower() for i in x])).value_counts()
    ----------------------------------------------------------------------------------------------------------
    """
    # Внутренняя функция, которая лупит конкретную строку.
    def transform(row, split_word, list_of_exceptions, replacement_dict):
        # Возвращаемый список.
        new_row = []
        # Цикл по элементам строки.
        for item in row:
            # Используется модуль re для поиска слова, которое окружено с двух сторон пробелами и проверяет его на вхождение в элемент списка.
            if re.search(r'\b' + split_word.lower() + r'\b', item.lower()):
                # Если это слова, по которым нужно именно разделить на отдельные части, то:
                if split_word.lower() == 'with' or split_word.lower() == 'and':
                    # бьём по нему элемент списка и
                    split_items = item.lower().split(split_word.lower())
                    # добавляем само слово разделитель и остальное, что осталось в разбитом элементе.
                    new_row.extend([split_word.strip(), *[i.strip() for i in split_items]])
                # Если же это другие слова или словосочетания, то:
                else:
                    # бьём,
                    split_items = item.lower().split(split_word.lower())
                    # добавляем само слово и склеиваем остатки.
                    new_row.extend([split_word.strip(), ' '.join([i.strip() for i in split_items])])
            # Если же слово не совпадает:
            else:
                # то просто добавляем элемент списка неизменным.
                new_row.append(item.strip())
        
        # Чистим строку.
        new_row = [item for item in new_row if len(item) > 2 and item not in list_of_exceptions]
        # Переименовываем элементы списка только тогда, когда это нужно и переименование зависит от слова-разделителя.
        if replacement_dict and split_word in replacement_dict:
            new_row = [replacement_dict[split_word] if split_word in item else item for item in new_row]

        return new_row
    
    # Общий прогресс можно убрать. Просто у меня файл достаточно большой и это занимает пару минут. Так чисто, декоративное.
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############
    # Общий прогресс прогресс бара от 0 до 100% и Инициализация прогресс бара с настройками отображения.
    total_progress = 100
    with tqdm_notebook(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s, Remaining: {remaining} m/s]') as pbar:
        cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100%.
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
        
        # Здесь сам цикл, в котором мы перебираем каждое слово и передаём его в каждую строку, которая является списком.))
        # И обрабатываем это слово с помощью функции transform для каждой строки.
        for i, word in enumerate(words_to_transform):
            tags_column = tags_column.apply(transform, split_word=word, list_of_exceptions=list_of_exceptions, replacement_dict=replacement_dict)
            
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############        
            # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
            remaining_iterations = len(words_to_transform) - i # вот тут пригодился индекс i
            remaining_progress = total_progress - cumulative_progress
            step_size = remaining_progress / remaining_iterations
            
            # Обновляем общий прогресс.
            cumulative_progress += step_size
            pbar.update(step_size)
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############

    return tags_column

[to contents](#contents)

<h3 id='multiplemodelstraining'>Функция для тренировки нескольких моделей, используя несколько алгоритмов ML за 1 заход</h3>

In [ ]:
def multiple_models_training(df, target_feature, list_of_ml_algs, params_dict=False, random_state=42, test_size=0.25):
    print(f'{Fore.RED}КОРОЛЬ АРТУР НА НАС НАПАЛИ!!!{Fore.RESET}') # https://www.youtube.com/watch?v=-XuyTXj_5F8&t=6s хз, чёт вспомнилось...
    """ Freeware by Akialema.
    Сие ваяние было вдохновлено... В телеграмме периодически запускают эвент типа мол: "попробуйте себя в роли датасаентиста",
    который длится 3 дня. И на третий день предлагалось сделать вот это вот, что происходит в проекте-3 и что происходит в этой функции: 
    х-трэйны, у-трэйны, тэсты, предсказания, кто там на что делится.. тыща параметров и т.д. И финальным заданием было попробовать 
    перебрать несколько алгоритмов, чтобы получить наименьшее некое число. И вот в тот момент я понял, что для этого дела нужен цикл.
    С графическим представлением. Потому что не даром их так много. А потом я посмотрел прекрасную работу Андрея Волкова и такой: 
    вот же она, эта картинка.) И чтоб не плодить простыню х-трэйнов-у-трэйнов, её можно просто обернуть в цикл и в функцию.
    Я сначала делал график прям в функции, чтоб такой ждёшь, ждёшь выполнения ячейки и хочется какой-то интерактивности прогресса 
    с последующим финальным представлением графика сравнения МАРЕ на сладкое. Но потом понял, что лучше делать картиночку отдельной ячейкой.
    Короче кайфанул и доволен. Да и я хз, может есть уже реализация подобного, я ж новичок. Потому быстро накалякал своё. Вообще не понимаю,
    что там происходит и правильно ли. Но работает. Я просто хорошо задаю вопросы чату гпт.))) 
    Забирайте, если понравилось.)
    
    P.S. функция не идеальна. Я не знаю названия всех алгоритмов. Какие чат гпт сказал, что топ, те и тут.  
    Явно я эту функцию ещё дополню. Или пойму, что это пустая трата времени. 
    P.S.2 ЕСЛИ ВЫ ЗНАЕТЕ, КАК СДЕЛАТЬ ПЛАВНЫЙ ПРОГРЕСС БАР ПО ЯЧЕЙКЕ - РАССКАЖИТЕ МНЕ. Пожалуйста. <^-------- Можно не читать.
    
    Функция проведения нескольких (или одной) тренировок моделей.
    
    ---> На вход подаём итоговый датафрейм, в котором только числа и нет нанок, 
         целевой признак,
         список названий алгоритмов для тренировки,
         и словарь с параметрами для алгоритмов.

    Args:
        df (pd.Dataframe): итоговый дф.
        target_feature (str): целевой признак.
        list_of_ml_algs (list): список Агрессоров, каждый из них в str формате.)) (агрессоров может быть как 1, так и сколько душе угодно, но важно, чтобы они соответствовали структуре "инит класса -> тренировка -> предсказание").
        params_dict (dict, optional): словарь вида {alg: [{параметры алгоритма}, {параметры фита}]}, по которому создаётся датафрейм с параметрами. Первый внутренний словарь для параметров алгоритма, второй - для фита. Defaults to False.
        random_state (int, optional): я хз, что это, я так до сих пор и не прочувствовал np.random.seed. Defaults to 42.
        test_size (float, optional): размер тестовой выборки. Defaults to 0.25.

    Returns:
        pd.Dataframe: На выход получаем датафрейм с натренированными моделями, числом МАРЕ и чем-нибудь ещё.
        
    Пример работы. Нужно скопировать код ниже и вставить в ячейку после функции.
    ------------------------------------------------------------------------------------------------------
    # Здесь нужно выбрать, какими алгоритмами будем гонять числа.)
    list_of_ml_algs = ['RandomForestRegressor', 'ExtraTreesRegressor', 'BaggingRegressor', 'GradientBoostingRegressor', 'CatBoostRegressor', 'AdaBoostRegressor', 'DecisionTreeRegressor']
    
    # Параметры для каждого алгоритма. Порядок не важен, как и указание параметров для всех алгоритмов. 
    # Можно указать параметры только для тех, для которых нужно, остальные отработают по своим дефолтным параметрам.
    # Важна структура {alg: [{параметры алгоритма}, {параметры фита}]}, иначе можно хлебнуть горя.)) Индекс-столбец-столбец.
    # А можно и не делать этот словарь вовсе. Алгоритмы отработают по своим настройкам по-умолчанию.
    params_dict = {
            'RandomForestRegressor': [{'n_estimators': 10, 'n_jobs': -1, 'random_state': 42}, {}],
            'ExtraTreesRegressor': [{'n_estimators': 10, 'max_depth': 3}, {}],
            'BaggingRegressor': [{'n_estimators': 10}, {}],
            'GradientBoostingRegressor': [{'n_estimators': 10, 'learning_rate': .1}, {}],
            'CatBoostRegressor': [{'iterations': 10000, 'learning_rate': .1, 'depth': 3, 'verbose': False}, {}],
            'AdaBoostRegressor': [{'n_estimators': 10, 'learning_rate': 1.0}, {}],
            'DecisionTreeRegressor': [{}, {}]
    } # везде эстиматоры 10, чтоб оно отработало быстро и показало работу. 
    
    # Здесь нужно вставить свой end_df, свой 'целевой_признак', решить, какими алгоритмами будем баловаться и с какими параметрами.)
    any_var_name = multiple_models_training(end_df, 'reviewer_score', list_of_ml_algs=list_of_ml_algs, params_dict=params_dict)
    
    any_var_name
    ------------------------------------------------------------------------------------------------------
    
    Нужно быть внимательным/ной, чтоб всё сработало, как надо. Я вроде всё расписал.
    """
    
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from colorama import Fore, Back, Style # Чтоб аутпут был цветастый.
    from tqdm.notebook import tqdm
    import pandas as pd
    import time
    
    # Если мы передаём параметры словарём извне.
    if params_dict:
        params_dict_func = {alg: [{}, {}] for alg in list_of_ml_algs} # Создаётся словарь следующей формы: {alg: [{}, {}]}, для каждого элемента списка с названиями алгоритмов.
        params_dict_func.update(params_dict) # Обновление словаря словарём извне.
        params_df = pd.DataFrame.from_dict(params_dict_func, orient='index', columns=['alg_params', 'fit_params']) # ДФ по словарю.
    # Если мы не передаём параметры в функцию, то все параметры будут по умолчанию для каждого алгоритма.
    else:
        params_df = pd.DataFrame([[{},{}]], index=list_of_ml_algs, columns=['alg_params', 'fit_params'])
    
    X = df.drop(columns=[target_feature], axis=1) # дропаем целевой признак из икса.
    y = df[target_feature] # и помещаем его в игрек.
    
    # Эти штуки.. Я надеюсь они в правильном порядке стоят. Вроде бы у всех так.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    list_of_algs_output = [] # список по которому будем формировать выходящий датафрейм.
    
    total_progress = 100  # Общий прогресс прогресс бара от 0 до 100%
    
    # Инициализация прогресс бара с настройками отображения.
    with tqdm(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s]') as pbar:
        
        cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100.
        
        # Цикл по списку алгоритмов.
        for i, alg in enumerate(list_of_ml_algs): # i пригодился в вычислении шага прогрессбара.
            
            start_alg_time = time.time() # Время начала работы обучения модели.
            
            # Инит алгоритма через globals, т.к. название алгоритма == str. (можно через eval(), но гпт чату не понравилась эта идея) 
            # Параметры == параметры столбца 'class_params' для строки с именем alg.
            model = globals()[alg](**params_df.loc[alg, 'alg_params']) 
            
            model.fit(X_train, y_train, **params_df.loc[alg, 'fit_params']) # Здесь сама тренировка. Параметры из столбца 'fit_params'.
            
            current_time = time.time() # Время окончания работы алгоритма.
            
            y_pred = model.predict(X_test) # Предсказание.
            
            # Делаем МАПЕ.
            mape = metrics.mean_absolute_percentage_error(y_test, y_pred) * 100
            # Я не знаю, что это и для чего это здесь. Просто добавил, чтобы убедиться, что в вывод можно ещё чего-нибудь прикрутить.
            # Соответственно, нужно обновлять аппенд и формирование столбцов итогового дф.
            accuracy = model.score(X_test, y_test) 
            
            list_of_algs_output.append([alg, mape, accuracy, model]) # Добавляем всю эту петрушку в список, по которому будет формироваться таблица.
            
            ##########################################################################################################################
            
            # Вычисление времени для принта ниже. Типа вот, мол, модель обучилась, поздравляю.
            execution_time_alg = current_time - start_alg_time
            # Можно прикрутить, что-то другое для принта.
            print(f"Model training completed! With: {str(round(execution_time_alg, 2)) + ' sec.' if execution_time_alg < 60 else str(round(execution_time_alg/60, 2)) + ' min.'} Alg: {Fore.CYAN}{alg}{Fore.RESET}.") # чтоб секунды были секундами, а минуты - минутами.
            time.sleep(0.2) # Сон я добавил, потому что сначала прогресс бар отрабатывает, а потом принт. А надо наоборот.)
            
            # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
            remaining_iterations = len(list_of_ml_algs) - i # вот тут пригодился индекс i
            remaining_progress = total_progress - cumulative_progress
            step_size = remaining_progress / remaining_iterations
            
            # Обновляем общий прогресс. Все округления происходят при инициализации прогрессбара.
            cumulative_progress += step_size
            pbar.update(step_size)
        
    func_df = pd.DataFrame(list_of_algs_output, columns=['alg_name', 'mape_percent', 'accuracy', 'model']).sort_values(by='mape_percent').reset_index(drop=True) # ДФ-чик 
    
    print(f'{Fore.GREEN}Mission accomplished{Fore.RESET}.')
    # Возвращаем дф, в котором мапе и сама модель. 
    return func_df

[to contents](#contents)

<h3 id='vadersentanalsec'>Функция для формирования датафрейма с определением тональности текста по алгоритму Вейдера.</h3>

In [ ]:
def vader_sentiment_analyser(df, targ_col):
    """Функция для формирования датафрейма с определением тональности текста по алгоритму Вейдера.
    
    Args:
        df (pd.DataFrame): incoming df
        targ_col (str): target column

    Returns:
        pd.DataFrame: df with neg, neu, pos, compound from Vader.
        
        looks like:
        __|targ_col_neg|targ_col_neu|targ_col_pos|targ_col_comp|
        0 |         0.1|         0.8|         0.2|         0.64|
        etc.
    """
    # Импорт необходимых библиотек. Библиотеки лучше выносить за рамки функции.
    import pandas as pd
    from tqdm.notebook import tqdm
    import nltk
    from nltk.sentiment import SentimentIntensityAnalyzer
    nltk.download('vader_lexicon')
    
    # Инит класса.
    sia = SentimentIntensityAnalyzer()
    # Словарь для записи результатов.
    res = {}
    
    total_progress = 100
    with tqdm(total=total_progress, desc=f"Cell progress: ", bar_format='{desc}: {percentage:.0f}%|{bar}| {n:.0f}/{total_fmt} [Elapsed: {elapsed} m/s, Remaining: {remaining} m/s]') as pbar:
        cumulative_progress = 0 # Чтобы прогресс бар работал, как надо и завершался всегда на 100.
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
        
        # Здесь сам цикл, в котором мы перебираем каждое слово и передаём его в каждую строку, которая является списком.))
        # И обрабатываем это слово с помощью функции transform для каждой строки.
        for i, text in enumerate(df[targ_col]):
            res[i] = sia.polarity_scores(text)
            
    ##########vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv##############        
            # Рассчитываем размер шага на основе оставшегося прогресса и оставшихся итераций.
            remaining_iterations = len(df) - i # вот тут пригодился индекс i
            remaining_progress = total_progress - cumulative_progress
            step_size = remaining_progress / remaining_iterations
            
            # Обновляем внутренний и общий прогресс.
            cumulative_progress += step_size
            pbar.update(step_size)
    ##########^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^##############
    # Переименование столбцов.
    new_columns = {col: f"{targ_col}_{col}" for col in res[0].keys()}
    # Возврат дф-ки с решейпом и переименованием столбцов.
    return pd.DataFrame(res).T.rename(columns=new_columns)

[to contents](#contents)

<h3 id='vadersentanalmain'>Функция для формирования датафрейма с определением тональности текста по алгоритму Вейдера (основная).</h3>

In [ ]:
def vader_sentiment_analysis(df, targ_col):
    """Функция для формирования датафрейма с определением тональности текста по алгоритму Вейдера.
    
    Args:
        df (pd.DataFrame): incoming df
        targ_col (str): target column

    Returns:
        pd.DataFrame: df with neg, neu, pos, compound from Vader.
        
        looks like:
        __|targ_col_neg|targ_col_neu|targ_col_pos|targ_col_comp|
        0 |         0.1|         0.8|         0.2|         0.64|
        etc.
    """
    
    # Импорт необходимых библиотек. Библиотеки лучше выносить за рамки функции.
    import pandas as pd
    from tqdm.notebook import tqdm
    import nltk
    from nltk.sentiment import SentimentIntensityAnalyzer
    nltk.download('vader_lexicon')
    
    # Инит класса.
    sia = SentimentIntensityAnalyzer()
    # Словарь для записи результатов.
    res = {}
    
    # Цикл по строкам датафрейма и запись анализа настроения в словарь.
    for i, text in enumerate(tqdm(df[targ_col], total=len(df))):
        res[i] = sia.polarity_scores(text)
    
    # Переименование столбцов.
    new_columns = {col: f"{targ_col}_{col}" for col in res[0].keys()}
    # Возврат дф-ки с решейпом и переименованием столбцов.
    return pd.DataFrame(res).T.rename(columns=new_columns)

[to contents](#contents)

<h3 id='robertapolarity'>Функция с алгоритмом Роберты на основе отзывов в твиттере.</h3>

In [ ]:
def roberta_polarity_scores(df, targ_col):
    """Функция для формирования датафрейма с определением тональности текста по алгоритму Роберты.
    
    Args:
        df (pd.DataFrame): incoming df
        targ_col (str): target column

    Returns:
        pd.DataFrame: df with neg, neu, pos, compound from Vader.
        
        looks like:
        __|targ_col_neg|targ_col_neu|targ_col_pos|
        0 |         0.1|         0.8|         0.2|
        etc.
    """
    # Подгрузка библиотек. Библиотеки лучше вынести за пределы функции.
    from transformers import AutoTokenizer
    from transformers import AutoModelForSequenceClassification
    from scipy.special import softmax
    
    # Инит классов, токенизеров.
    MODEL = f'cardiffnlp/twitter-roberta-base-sentiment'
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    
    # Словарь, по которому формируется выходящий дф.
    res = {}
    # Цикл по строкам столбца.
    for i, text in enumerate(tqdm(df[targ_col], total=len(df))):
      # Трай - эксепт здесь для того, что некоторые строки слишком длинные и алгоритм роберты с ними не справляется и выдаёт ошибку.
      try: 
        # Всё, что ниже можно написать в 1 строку, но го в 4.
        encoded_text = tokenizer(text, return_tensors='pt') # Тут токенизация слов.
        output = model(**encoded_text) # Тут я не помню, какой аутпут был, если интересно - ниже в одну строку и можно обрубить, и посмотреть на каждый этап.
        scores = softmax(output[0][0].detach().numpy()) # Здесь происходит экспоненцирование (вот такое вот слово) чисел с помощью softmax.
        res[i] = {'roberta_neg': scores[0], 'roberta_neu': scores[1], 'roberta_pos': scores[2]} # Ну и запись в словарь.
      except RuntimeError:
        print(f'Broke for id: {i}') # если ошибка, то выводим номер строки.

    # Переименование столбцов.
    new_columns = {col: f"{targ_col}_{col}" for col in res[0].keys()}
    # Возврат дф-ки с решейпом и переименованием столбцов.
    return pd.DataFrame(res).T.rename(columns=new_columns)

# Эта строчка выведет настроение конкретной строки текста, т.е. вместо 'example' нужно вставить 'любой текст'.
#print(softmax(AutoModelForSequenceClassification.from_pretrained(f'cardiffnlp/twitter-roberta-base-sentiment')(**AutoTokenizer.from_pretrained(f'cardiffnlp/twitter-roberta-base-sentiment')('example', return_tensors='pt'))[0][0].detach().numpy())) # вместо 'example' любой текст

[to contents](#contents)

<h3 id='strongcolssearcher'>Функция для автоматического поиска сильных столбцов относительно целевого столбца (тестовая).</h3>

In [ ]:
def strong_cols_auto_searcher(df, list_of_cat_cols, list_of_num_cols, list_of_agg_func, targ_col, threshold_number=4000):
    import pandas as pd
    import numpy as np
    from sklearn.feature_selection import f_classif
    from tqdm.notebook import tqdm
    
    """strong_cols_auto_searcher:

    Функция, которая ищет наиболее значимые столбцы, для предсказания целевого столбца.
    т.е. по очереди происходит группировка всех названий столбцов из списка list_of_cat_cols с столбцами из списка list_of_num_cols
    с агрегацией функций из списка list_of_agg_func и сразу происходит отсев по пороговому числу threshold_number.
    Ну и запись в выходящий датафрейм.

    Args:
        df (pd.DataFrame): incoming df.
        list_of_cat_cols (list): [['col1'], ['col2'], ['col3', 'col4']] list with lists inside with str in this lists.
        list_of_num_cols (list): ['col1', 'col2', 'col3'] just list with str elements.
        list_of_agg_func (list): ['mean', 'count', 'nunique', 'sum'] list with any func name.
        targ_col (str): 'col' - any col u like.
        threshold_number (int, optional): minimum for f_classif. if < then we dont need this column. Defaults to 4000.

    Returns:
        pd.DataFrame: df with data about what and by what was groupped with what function for each column at the last row of each column. 
    """
    # Копия для основной таблицы.
    main_df = df.fillna(0).copy()
    # Номер последней строки, куда потом будет происходить запись что с чем группируем и по какой функции.
    num_for_new_row = main_df.shape[0]
    
    # Пустой итоговый дф.
    func_dict = {}
    annot_dict = {}
    
    # Цикл по тому, по чему группируем. С добавлением прогресс бара.
    for i in tqdm(list_of_cat_cols, total=len(list_of_cat_cols)):
        # Цикл по тому, что группируем. 
        for j in list_of_num_cols:
            
            # Если столбец есть и там, и там, то пропускаем такую группировку.
            if j in i:
                continue
            
            # Если тип столбца для группировки О (например там столбец с строками)
            if np.issubdtype(main_df[j], 'object'):
                # То применяем только count, т.к. mean не проканает, а sum - бессмысленен.
                func_series = main_df.groupby(i)[j].transform('count') ############ здесь слабое место, я пока не придумал, как его обыграть.
                # Присваиваем этому срезу новое название.
                func_series.name = 'strong_column_' + str(len(func_dict))
                # Далее происходит запись в словари. Записывается сам сериес и данные по сериесу, каждая запись в свой словарь.
                func_dict[func_series.name] = func_series.to_dict()
                annot_dict[func_series.name] = {f"'cat_cols': {i}, 'num_cols': {j}, 'func': {k}"}
            
            # Если же тип столбца для группировки числовой:
            else:            
                # Цикл по функциям.
                for k in list_of_agg_func:
                    # И дальше всё то же самое. 
                    # Сгруппировали, переименовали, записали в словари.
                    func_series = main_df.groupby(i)[j].transform(k)
                    func_series.name = 'strong_column_' + str(len(func_dict))
                    
                    func_dict[func_series.name] = func_series.to_dict()
                    annot_dict[func_series.name] = {f"'cat_cols': {i}, 'num_cols': {j}, 'func': {k}"}
    
    # Формируем датафрейм по основному словарю.
    func_df = pd.DataFrame(func_dict)
    # Отсеиваем лишние столбцы, согласно условию, что число теста будет больше заданного нами, что определяет силу чтолбца.
    func_df = func_df[func_df.columns[pd.Series(f_classif(func_df, main_df[targ_col].astype(int))[0]) > threshold_number]]
    # Добавляем описание в последнюю строку из словаря annot_dict.
    # Сначала формируем дф по нему, решейпим и конкатим.
    func_df = pd.concat([func_df, pd.Series(annot_dict).to_frame().T], axis=0, join='inner').reset_index(drop=True)
        
    return func_df

# Пример работы:
#list_of_cat_cols = [['hotel_town', 'year_of_review', 'quarter_of_review', 'hotel_name'], ['hotel_town', 'year_of_review', 'hotel_name']]
#list_of_num_cols = ['reviewer_score']
#list_of_agg_func = ['mean']
#targ_col = 'reviewer_score'
#
#strong_cols_auto_searcher(booking_df, list_of_cat_cols=list_of_cat_cols, list_of_num_cols=list_of_num_cols, list_of_agg_func=list_of_agg_func, targ_col=targ_col)

[to contents](#contents)

<h3 id='combinationsgenerator'>Функция для генерации комбинаций элементов входящего списка.</h3>

In [1]:
def generate_all_combinations(incoming_list, critical_num=1):
    """generate_all_combinations 

    Функция принимает на вход список и возвращает список скомбинированных списков.
    Например:
    вход ['a', 'b', 'c'] -> выход [['a'], ['b'], ['c'], ['a', 'b'], ['a', 'c'], ['b', 'c'], ['a', 'b', 'c']]

    Args:
        incoming_list (list): incoming list of something.
        critical_num (int, optional): thats if we dont need all combinations, just first half. Defaults to 1.
        
    Returns:
        list: list with all combinations of items.
    """
    
    # Внутренняя функция рекурсивная и делает она следующее:
    # На каждый length, например пример с ['a', 'b', 'c'] на входе:
    # При length=1 на выходе получаем [['a'], ['b'], ['c']]
    # При length=2 - [['a', 'b'], ['a', 'c'], ['b', 'c']]
    # При length=3 - [['a', 'b', 'c']]
    def generate_combinations(incoming_list, length):
        if length == 1:
            return [[item] for item in incoming_list]
        else:
            combinations = []
            for i, item in enumerate(incoming_list):
                for next_combination in generate_combinations(incoming_list[i+1:], length - 1):
                    combinations.append([item] + next_combination)
            return combinations
    
    # И ниже мы это всё безобразие записываем в выходящий список.
    # А критикал нум нам нужен для того, чтобы если вдруг нам не надо все все комбинации, а, например, половину...
    all_combinations = []
    for length in range(1, len(incoming_list) // critical_num + 1):
        all_combinations.extend(generate_combinations(incoming_list, length))
    return all_combinations

[to contents](#contents)

<h3 id='highcorreraser'>Функция для удаления сильно скоррелированных столбцов. (тестовая)</h3>

In [ ]:
def high_corr_eraser(df, upper_bound=0.9, lower_bound=-0.9, thresh=1, method='pearson'):
    
    func_df = df.copy()
    
    corr_df = func_df.corr(numeric_only=True, method=method)
    
    list_of_low_corr_columns_name = corr_df.corr(numeric_only=True)[(corr_df.corr(numeric_only=True) <= upper_bound) | ((corr_df.corr(numeric_only=True) >= lower_bound) & (corr_df.corr(numeric_only=True) <= 0))].dropna(thresh=thresh, axis=1).columns
    
    return func_df[list_of_low_corr_columns_name]

[to contents](#contents)

<h3 id='fclassifchitester'>Функция для тестирования силы столбца, относительно целевого. f_classif + chi2</h3>

In [ ]:
def f_classif_chi_tester(series, targ_col):
    """f_classif_chi_tester _summary_

    Функция.. проводит тестирование f_classifom и хи квадратом. И показывает число. Чем больше - тем лучше.
    Хи квадрат какой-то левый тест. Многомилионные показатели никак не влияют на модель. Как будто там что-то ломается.
    Но не суть. f_classif более правдивый.

    Args:
        series (pd.Series/pd.DataFrame): incoming series num only. Or if its df
        targ_col (pd.Series): looks like your_df['your_targ_col']

    Returns:
        _type_: _description_
    """
    import pandas as pd
    from sklearn.feature_selection import chi2 # хи-квадрат
    from sklearn.feature_selection import f_classif # anovav
    
    func_series_or_df = series.copy()
    
    # if series
    if isinstance(func_series_or_df, pd.Series):
        chi = pd.Series(chi2(func_series_or_df.to_frame().abs(), targ_col.astype(int))[0], index=['chi'])
        f = pd.Series(f_classif(func_series_or_df.to_frame(), targ_col.astype(int))[0], index=['f'])
    
    else: # if df
        if targ_col.name in func_series_or_df.columns: # drop targ_col if it exists.
            func_series_or_df.drop(columns=targ_col.name, inplace=True)
        chi = pd.Series(chi2(func_series_or_df.abs(), targ_col.astype(int))[0], index=func_series_or_df.columns, name='chi')
        f = pd.Series(f_classif(func_series_or_df, targ_col.astype(int))[0], index=func_series_or_df.columns, name='f_classif')
        
        return pd.DataFrame([f, chi]).T
    return pd.concat([f, chi])

[to contents](#contents)

<h3 id='specialgrouper'>Группировщик с применением ко всему датафрейму</h3>

In [ ]:
def special_grouper(df, bywhat, what, list_of_agg_func=['sum', 'mean', 'count', 'nunique']):
    """special_grouper 

    Согласитесь, бесит, когда надо писать несколько строчек, чтобы группировка применилась ко всей таблице.
    А метод transform принимает только 1 аргумент и ему плевать список там или нет. Хотя в документации там может быть список.
    Этот момент я до конца не понял.
    
    Эта функция - исключительно для визуальной оценки, а потом мы делаем нужный нам сериес с выбранной агрегацией через transform.
    Т.к. трансформ крутая штука на самом деле, туда можно пихнуть лямбду.
    
    Хотя и отсюда можно взять столбец, но через .loc/iloc, потому что функция возвращает тип датафрейм.

    Args:
        df (pd.DataFrame): any df.
        bywhat (str or list): name or names of columns which we want to group.
        what (str): name of col we wanna group bywhat (my logic works only for single col cuz .iloc[bla bla] (can be expanded))
        list_of_agg_func (list, optional): list of agg func. Defaults to ['sum', 'mean', 'count', 'nunique'].

    Returns:
        pd.DataFrame: a column or columns with grouping applied to the entire df as a df.
    """
    return pd.merge(df[bywhat], df.groupby(bywhat)[what].agg(list_of_agg_func).reset_index(), on=bywhat, how='left').iloc[:, -len(list_of_agg_func):]

[to contents](#contents)

<h3 id='goodbadwordssearcher'>Функция для определения положительных и отрицательных слов в предложении</h3>

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# Функция для определения тональности слова
def extract_sentiment_words(sentence, positive_words, negative_words):
    
    def get_word_sentiment(word, positive_words, negative_words):
        # Создание синтаксических представлений для положительных и отрицательных слов    
        positive_synsets = [j for i in positive_words for j in wordnet.synsets(i, pos=wordnet.ADJ)]
        negative_synsets = [j for i in negative_words for j in wordnet.synsets(i, pos=wordnet.ADJ)]
        
        # Определение синонимов слова в WordNet
        synsets = wordnet.synsets(word, pos=wordnet.ADJ)
        if synsets:
            # Проверка синонимов на положительность или отрицательность
            for synset in synsets:
                if synset in positive_synsets:
                    return 'positive'
                elif synset in negative_synsets:
                    return 'negative'
        # Если не удалось определить тональность
        return 'neutral'
    
    positive_list = []
    negative_list = []
    words = nltk.word_tokenize(sentence)
    for word in words:
        sentiment = get_word_sentiment(word, positive_words, negative_words)
        if sentiment == 'positive':
            positive_list.append(word)
        elif sentiment == 'negative':
            negative_list.append(word)
    return positive_list, negative_list

sentense = 'i ok feel very conflicted about this hotel unfortunately ultimately i wouldn t recommend it because with our experience the bad outweighed the good the fire alarm went off late on sunday night 11th july on the first night we stayed so naturally everyone evacuated and stood outside it was absolutely pouring so not the best start eventually a member of hotel staff came to the front desk and the alarm was silenced i would say the alarm had been going for about 10 minutes at this point though and there wasn t a staff member to tell us if it was a false alarm or real or anything she explained in a really aggressive manner that there was a fault with the alarm and heavy rain sometimes set it off she seemed irritated we had bothered to evacuate and said that it was not the first time it had happened no apology or further explanation of a fix or anything fair enough though it was close to 1am and we were all a bit moody we arrived back at the hotel on our first day and the girl at the front desk informed us the lift was broken we were on the top floor which was a pain but being able bodied it was hardly a big deal however it was not fixed for the full 4 days of our stay again no apology offered at any point they did however offer to help getting luggage downstairs which was a godsend no idea what they would have done if they had disabled guests all in all the staff were unpleasant to deal with when we returned to the hotel in the evening we arrived home late from disneyland on tuesday night and asked for some milk sachets for the room and we were told no milk is available by an exasperated girl on the front desk who was busy making a personal call we walked up the four flights of stairs and found some milk sachets on a cleaning trolley on the landing so i think she just had no interest in being helpful also just a word of advice our room 405 had the wc seperate then a shower in the middle of the bedroom'

# Определение положительных и отрицательных слов
positive_words = ['good', 'great', 'excellent', 'ok', 'awesome']  # Положительные слова
negative_words = ['bad', 'poor', 'awful', 'conflict']  # Отрицательные слова

print(extract_sentiment_words(sentence=sentense, positive_words=positive_words, negative_words=negative_words))
print(wordnet.synsets('ok', pos=wordnet.ADJ))